In [17]:
import langchain
langchain.debug = True
import base64
import requests
from typing import Optional, Dict, Any, Union
import os
def make_request(
    method: str,
    url: str,
    headers: Optional[Dict[str, str]] = None,
    params: Optional[Dict[str, Union[str, int, float]]] = None,
    data: Optional[Dict[str, Any]] = None,
    json: Optional[Dict[str, Any]] = None,
    timeout: Optional[int] = 30
) -> requests.Response:
    """
    Make a generic HTTP request.
    
    :param method: HTTP method (e.g., GET, POST, PUT, DELETE)
    :param url: The URL for the request
    :param headers: Optional headers to include in the request
    :param params: Optional query parameters
    :param data: Data to send in the body of the request (for POST/PUT)
    :param json: JSON data to send in the body of the request (for POST/PUT)
    :param timeout: Request timeout in seconds
    :return: `requests.Response` object
    """

    supported_methods = {
        'GET': requests.get,
        'POST': requests.post,
        'PUT': requests.put,
        'DELETE': requests.delete
    }
    
    if method not in supported_methods:
        raise ValueError(f"Unsupported HTTP method: {method}")

    response = supported_methods[method](
        url,
        headers=headers,
        params=params,
        data=data,
        json=json,
        timeout=timeout
    )

    return response


In [18]:
#start a session
domain="http://localhost:5002"
url = f"{domain}/api/v1/jupyter/start"
response = make_request(
    method='POST', 
    url=url,
    headers={"accept":"application/json"},
    params={
        "token":"zhejianzhang",
        "language":"JUPYTER"
    }
    )
session_id=response.text.strip('"')

In [19]:
print(session_id)
print(type(session_id))

a71191f2-8568-443a-a011-dd0433003c85
<class 'str'>


In [20]:
import json


def execute(code):
    url = f"{domain}/api/v1/jupyter/execute_code"
    data = {
                "code": {
                "code": code
                },
                "session_key": {
                    "session_id":session_id,
                    "language": "JUPYTER"
                }
            }
    response = make_request(
        method='POST', 
        url=url,
        headers={"accept":"application/json",'Content-Type': 'application/json'},
        json=data
        )

    print(response)
    print('Execute Result=',response.text)
    return response.text

def install(packages):
    url = f"{domain}/api/v1/jupyter/install"
    data = {
                "install_request": {
                "packagenames":packages 
                },
                "session_key": {
                    "session_id":session_id,
                    "language": "JUPYTER"
                }
            }
    response = make_request(
        method='POST', 
        url=url,
        headers={"accept":"application/json",'Content-Type': 'application/json'},
        json=data
        )

    print(response)
    print('Execute Result=',response.text)
    



In [21]:
packages=['numpy','pandas','docker']
c = """
import matplotlib.pyplot as plt
import time

# 数据
x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

# 创建折线图
plt.plot(x, y, marker='o')

# 添加标题和轴标签
plt.title("Simple Line Plot")
plt.xlabel("X Axis")
plt.ylabel("Y Axis")

# 显示图形
plt.grid(True)  # 显示网格线
plt.show()
plt.savefig('test1.png')
x = [1, 2, 3, 4, 5]
y = [1, 4, 9, 16, 25]

# 创建折线图
plt.plot(x, y, marker='o')

# 添加标题和轴标签
plt.title("Simple Line Plot")
plt.xlabel("X Axis")
plt.ylabel("Y Axis")

# 显示图形
plt.grid(True)  # 显示网格线
plt.show()
plt.savefig('test2.png')

for i in x:
    print(i)
    time.sleep(1)
"""

install(packages)

<Response [200]>
Execute Result= "installed"


In [22]:
test_code="""
import docker
print(docker.__version__)
"""
execute(test_code)

<Response [200]>
Execute Result= {"output_type":"text/plain","content":"6.1.3\n","files":null}


'{"output_type":"text/plain","content":"6.1.3\\n","files":null}'

In [23]:
rsp=execute(c)
rsp = json.loads(rsp)
for item in rsp['files']:
    with open(item['name'], 'wb') as f:
        f.write(base64.b64decode(item['content']))


<Response [200]>
Execute Result= {"output_type":"files/mixed","content":"1\n\n2\n\n3\n\n4\n\n5\n","files":[{"name":"cb3900a6575811ee82cb0242ac160002_0.png","content":"..."},{"name":"cb3900a6575811ee82cb0242ac160002_1.png","content":"..."}]}
